In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
# csvファイルからPandas DataFrameへ読み込み
train = pd.read_csv('train.csv', delimiter=',', low_memory=False)
test = pd.read_csv('test.csv', delimiter=',', low_memory=False)

# 冒頭を表示して確認
train.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [3]:
# trainデータを入力データとラベルに分割する
X = train.drop(['date_time'], axis=1).drop(['target_carbon_monoxide'], axis=1).drop(['target_benzene'], axis=1).drop(['target_nitrogen_oxides'], axis=1).values
Y_1 = train.target_carbon_monoxide.values  # carbon_monoxide(一酸化炭素)
Y_2 = train.target_benzene.values          # benzene(ベンゼン)
Y_3 = train.target_nitrogen_oxides.values  # nitrogen_oxides(窒素酸化物)

# 表示して確認
X, Y_1

(array([[1.3100e+01, 4.6000e+01, 7.5780e-01, ..., 1.0560e+03, 1.7428e+03,
         1.2934e+03],
        [1.3200e+01, 4.5300e+01, 7.2550e-01, ..., 1.1975e+03, 1.4499e+03,
         1.0109e+03],
        [1.2600e+01, 5.6200e+01, 7.5020e-01, ..., 1.0602e+03, 1.5861e+03,
         1.1170e+03],
        ...,
        [9.6000e+00, 3.4600e+01, 4.3100e-01, ..., 8.6190e+02, 8.8920e+02,
         1.1591e+03],
        [8.0000e+00, 4.0700e+01, 4.0850e-01, ..., 9.0850e+02, 9.1700e+02,
         1.2063e+03],
        [8.0000e+00, 4.1300e+01, 4.3750e-01, ..., 7.9710e+02, 8.8000e+02,
         1.2731e+03]]),
 array([2.5, 2.1, 2.2, ..., 1.6, 1.5, 1.4]))

In [4]:
# データを訓練データと検証データに分割する
def split_data(X, Y):
    # trainデータを訓練データと検証データに分割する
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    
    return x_train, x_test, y_train, y_test

In [5]:
# RMSLE カスタム評価関数 #####################
from keras import backend as K
msle = keras.metrics.MeanSquaredLogarithmicError()

def root_mean_squared_logarithmic_error(y_true, y_pred):
    return K.sqrt(msle(y_true, y_pred))

In [6]:
# RNN(LSTM)モデルをセット
def model_set():
    # RNN(LSTM)モデルを構築
    model = keras.Sequential([
            keras.layers.LSTM(10, batch_input_shape=(None, 8, 1), return_sequences=True),
            keras.layers.Dense(10, activation='relu'),
            keras.layers.Dense(1)
    ])
    
    # モデルをコンパイルして構造を表示
    model.compile(optimizer='adam', #rmsprop
                  loss='mean_squared_error')
    model.summary()
    
    return model

# carbon_monoxide(一酸化炭素)について------------------------

In [7]:
# データを訓練データと検証データに分割する
x_train, x_test, y_train, y_test = split_data(X, Y_1)

# 形を表示して確認
((x_train.shape, x_test.shape), (y_train.shape, y_test.shape))

(((5688, 8), (1423, 8)), ((5688,), (1423,)))

In [8]:
# RNN(LSTM)モデルをセット
model_1 = model_set()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 8, 10)             480       
_________________________________________________________________
dense (Dense)                (None, 8, 10)             110       
_________________________________________________________________
dense_1 (Dense)              (None, 8, 1)              11        
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 学習させる
model_1.fit(x_train.reshape(-1, 8, 1), y_train, batch_size=256, epochs=100)

Epoch 1/100
23/23 [==============================] - 2s 4ms/step - loss: 5.4803
Epoch 2/100
23/23 [==============================] - 0s 4ms/step - loss: 4.6601
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 3.5566
Epoch 4/100
23/23 [==============================] - 0s 5ms/step - loss: 2.7107
Epoch 5/100
23/23 [==============================] - 0s 4ms/step - loss: 2.4137
Epoch 6/100
23/23 [==============================] - 0s 4ms/step - loss: 2.2484
Epoch 7/100
23/23 [==============================] - 0s 5ms/step - loss: 2.1389
Epoch 8/100
23/23 [==============================] - 0s 4ms/step - loss: 2.1761
Epoch 9/100
23/23 [==============================] - 0s 5ms/step - loss: 2.0552
Epoch 10/100
23/23 [==============================] - 0s 4ms/step - loss: 2.1098
Epoch 11/100
23/23 [==============================] - 0s 4ms/step - loss: 2.1917
Epoch 12/100
23/23 [==============================] - 0s 4ms/step - loss: 2.1648
Epoch 13/100
23/23 [=================

In [10]:
# 検証データへの精度
y_test_pred = model_1.predict(x_test.reshape(-1, 8, 1))[:,0]
print('RMSLE: %.3f' % root_mean_squared_logarithmic_error(y_test, y_test_pred))
print('r2: %.3f' % r2_score(y_test, y_test_pred))

RMSLE: 0.439
r2: -0.006


# benzene(ベンゼン)について---------------------------------

In [11]:
# データを訓練データと検証データに分割する
x_train, x_test, y_train, y_test = split_data(X, Y_2)

# 形を表示して確認
((x_train.shape, x_test.shape), (y_train.shape, y_test.shape))

(((5688, 8), (1423, 8)), ((5688,), (1423,)))

In [12]:
# RNN(LSTM)モデルをセット
model_2 = model_set()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 8, 10)             480       
_________________________________________________________________
dense_2 (Dense)              (None, 8, 10)             110       
_________________________________________________________________
dense_3 (Dense)              (None, 8, 1)              11        
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 学習させる
model_2.fit(x_train.reshape(-1, 8, 1), y_train, batch_size=256, epochs=100)

Epoch 1/100
23/23 [==============================] - 2s 11ms/step - loss: 165.5959
Epoch 2/100
23/23 [==============================] - 0s 6ms/step - loss: 159.5316
Epoch 3/100
23/23 [==============================] - 0s 5ms/step - loss: 152.8209
Epoch 4/100
23/23 [==============================] - 0s 5ms/step - loss: 155.6153
Epoch 5/100
23/23 [==============================] - 0s 5ms/step - loss: 132.0096
Epoch 6/100
23/23 [==============================] - 0s 5ms/step - loss: 107.1552
Epoch 7/100
23/23 [==============================] - 0s 5ms/step - loss: 78.3705
Epoch 8/100
23/23 [==============================] - 0s 5ms/step - loss: 63.6323
Epoch 9/100
23/23 [==============================] - 0s 5ms/step - loss: 63.8551
Epoch 10/100
23/23 [==============================] - 0s 5ms/step - loss: 62.8322
Epoch 11/100
23/23 [==============================] - 0s 5ms/step - loss: 60.8861
Epoch 12/100
23/23 [==============================] - 0s 5ms/step - loss: 62.3644
Epoch 13/100
23/23

23/23 [==============================] - 0s 7ms/step - loss: 45.3279


In [14]:
# 検証データへの精度
y_test_pred = model_2.predict(x_test.reshape(-1, 8, 1))[:,0]
print('RMSLE: %.3f' % root_mean_squared_logarithmic_error(y_test, y_test_pred))
print('r2: %.3f' % r2_score(y_test, y_test_pred))

RMSLE: 0.631
r2: 0.010


# nitrogen_oxides(窒素酸化物)について-------------------------

In [15]:
# データを訓練データと検証データに分割する
x_train, x_test, y_train, y_test = split_data(X, Y_3)

# 形を表示して確認
((x_train.shape, x_test.shape), (y_train.shape, y_test.shape))

(((5688, 8), (1423, 8)), ((5688,), (1423,)))

In [16]:
# RNN(LSTM)モデルをセット
model_3 = model_set()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 8, 10)             480       
_________________________________________________________________
dense_4 (Dense)              (None, 8, 10)             110       
_________________________________________________________________
dense_5 (Dense)              (None, 8, 1)              11        
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________


In [17]:
# 学習させる
model_3.fit(x_train.reshape(-1, 8, 1), y_train, batch_size=256, epochs=100)

Epoch 1/100
23/23 [==============================] - 4s 8ms/step - loss: 76748.0924
Epoch 2/100
23/23 [==============================] - 0s 7ms/step - loss: 78036.1751
Epoch 3/100
23/23 [==============================] - 0s 6ms/step - loss: 80818.9655
Epoch 4/100
23/23 [==============================] - 0s 8ms/step - loss: 79987.0472
Epoch 5/100
23/23 [==============================] - 0s 8ms/step - loss: 80011.3262
Epoch 6/100
23/23 [==============================] - 0s 8ms/step - loss: 76498.4427
Epoch 7/100
23/23 [==============================] - 0s 7ms/step - loss: 75585.2617
Epoch 8/100
23/23 [==============================] - 0s 6ms/step - loss: 75977.4085
Epoch 9/100
23/23 [==============================] - 0s 6ms/step - loss: 74401.8385
Epoch 10/100
23/23 [==============================] - 0s 5ms/step - loss: 76054.3076
Epoch 11/100
23/23 [==============================] - 0s 6ms/step - loss: 72609.7201
Epoch 12/100
23/23 [==============================] - 0s 6ms/step - loss: 

23/23 [==============================] - 0s 6ms/step - loss: 38542.5099
Epoch 98/100
23/23 [==============================] - 0s 5ms/step - loss: 40093.2573
Epoch 99/100
23/23 [==============================] - 0s 5ms/step - loss: 37668.8944
Epoch 100/100
23/23 [==============================] - 0s 5ms/step - loss: 39544.2078


In [18]:
# 検証データへの精度
y_test_pred = model_3.predict(x_test.reshape(-1, 8, 1))[:,0]
print('RMSLE: %.3f' % root_mean_squared_logarithmic_error(y_test, y_test_pred))
print('r2: %.3f' % r2_score(y_test, y_test_pred))

RMSLE: 0.733
r2: -0.044


# 提出データへの適用

In [19]:
# testデータを適用できる形に変換する
X_sb = test.drop(['date_time'], axis=1).values

# 表示して確認
X_sb

array([[8.0000e+00, 4.1300e+01, 4.3750e-01, ..., 7.9710e+02, 8.8000e+02,
        1.2731e+03],
       [5.1000e+00, 5.1700e+01, 4.5640e-01, ..., 6.8790e+02, 9.7280e+02,
        1.7140e+03],
       [5.8000e+00, 5.1500e+01, 4.6890e-01, ..., 6.9370e+02, 9.4190e+02,
        1.3008e+03],
       ...,
       [2.6600e+01, 1.9000e+01, 6.4060e-01, ..., 5.7280e+02, 1.2534e+03,
        1.0811e+03],
       [2.9100e+01, 1.2700e+01, 5.1390e-01, ..., 7.0200e+02, 1.0098e+03,
        8.0850e+02],
       [2.7900e+01, 1.3500e+01, 5.0280e-01, ..., 6.0820e+02, 1.0613e+03,
        8.1600e+02]])

In [20]:
# 各目的変数を回帰
predict_1 = pd.DataFrame(model_1.predict(X_sb.reshape(-1, 8, 1))[:,0])
predict_2 = pd.DataFrame(model_2.predict(X_sb.reshape(-1, 8, 1))[:,0])
predict_3 = pd.DataFrame(model_3.predict(X_sb.reshape(-1, 8, 1))[:,0])

# データの形を確認
predict_1.shape, predict_2.shape, predict_3.shape

((2247, 1), (2247, 1), (2247, 1))

In [21]:
# 結果を提出形式に変形
df_predictions = pd.concat([predict_1, predict_2, predict_3],axis=1)
df_predictions.columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
submit_data = pd.concat([test.date_time, df_predictions],axis=1)
submit_data = submit_data.set_index('date_time')
submit_data.head()

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
date_time,,,
2011-01-01 00:00:00,1.961489,9.071247,160.626053
2011-01-01 01:00:00,1.756637,8.429765,156.717758
2011-01-01 02:00:00,1.826213,8.614717,158.063019
2011-01-01 03:00:00,1.743638,8.400885,156.491013
2011-01-01 04:00:00,1.672816,8.244759,155.186630


In [22]:
# CSVファイルとして出力
submit_data.to_csv("submission_RNN_LSTM.csv")